In [ ]:
#!pip install scikit-learn==1.5.2
# run this line without the '!' in your terminal to have it installed locally
# then you can skip running this



# create days_since_april_1 column
# convert dateTime to days out of 365
# for each value, add 365, subtract april 1st as a number of days
# for each value, mod by 365

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   --------------- ------------------------ 4.2/11.0 MB 25.1 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 28.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/43.6 MB ? eta -:--:--
   ------ --------------------------------- 7.3/43.6 MB 36.6 MB/s eta 0:00:01
   ------------- -------------------------- 14.9/43.6 MB 36.7 MB/s eta 0:00:01
   -------------------- ------------------- 22.0/43.6 MB 35.6 MB/s eta 0:00:01
   -------------------------- ------------- 29.1/43.6 MB 35.0 MB/s eta 0:00:01
   --------------------------------- ------ 36.7/43.6 MB 35.2 MB/s eta 0:00:01
   ---------------------------------------  43.5/43.6 MB 34.9 MB/s eta 0:00:01
   ---------------------------------------- 43.6/43.6 MB 32.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Python313\python.exe -m pip install --upgrade pip


In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Load dataset
#housing = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv', dtype={'id': str})
# Load the holdout dataset
#holdout_data = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing_holdout_test_mini.csv', dtype={'id': str})

In [2]:
# Load dataset
housing = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv', dtype={'id': str})

# seperating id column to match major and minor.
housing['Major'] = housing['id'].str[:-4]
housing['Minor'] = housing['id'].str[-4:].astype(int).astype(str)
# now combining again, this is to get rid of trailing and leading zeros to elimate more duplicates
housing['more'] = housing['Major'].astype(str) + 'm' + housing['Minor'].astype(str)

# now inner joing county data set with training or prediction data set
df2 = pd.read_csv("C://git//ml//housing_project//testcode//merged_data_5.csv", dtype={'more': str})

housing['more'] = housing['more'].astype(str)
df2['more'] = df2['more'].astype(str)

housing = housing.merge(df2, on='more', how='inner')



y = np.log1p(housing["price"])  # Log transform target

X = housing.drop(["price", "id", "date", "Major_x", "Minor_x", "more", "AcctNbr", "Major_y", "Minor_y", "ZipCode", "SqFtLot"], axis=1)
X = pd.get_dummies(X)

In [3]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19957 entries, 0 to 19956
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bedrooms              19957 non-null  int64  
 1   bathrooms             19957 non-null  float64
 2   sqft_living           19957 non-null  int64  
 3   sqft_lot              19957 non-null  int64  
 4   floors                19957 non-null  float64
 5   waterfront            19957 non-null  int64  
 6   view                  19957 non-null  int64  
 7   condition             19957 non-null  int64  
 8   grade                 19957 non-null  int64  
 9   sqft_above            19957 non-null  int64  
 10  sqft_basement         19957 non-null  int64  
 11  yr_built              19957 non-null  int64  
 12  yr_renovated          19957 non-null  int64  
 13  zipcode               19957 non-null  int64  
 14  lat                   19957 non-null  float64
 15  long               

In [4]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create XGBoost regressor
model = xgb.XGBRegressor(objective="reg:squarederror", eval_metric="rmse", subsample=0.6,
                         n_estimators=1700, max_depth=6, learning_rate=0.01, colsample_bytree=0.7)

# Train model
model.fit(X_train, y_train)

# Make predictions on test set
y_pred = model.predict(X_test)

In [5]:
# Evaluate performance
y_non_log_test = np.expm1(y_test)
y_non_log_pred = np.expm1(y_pred)

mse = mean_squared_error(y_non_log_test, y_non_log_pred)
rmse = np.sqrt(mse)  # Compute RMSE
# return from a log of cost to cost
# rmse = np.expm1(rmse)
r2 = r2_score(y_test, y_pred)  # Compute R^2
r2_2 = r2_score(y_non_log_test, y_non_log_pred)  # Compute R^2

print(f"R² Score: {r2:.4f} {r2_2:.4f}")
print(f"RMSE: {rmse:.4f}")

R² Score: 0.9223 0.9315
RMSE: 92759.5619


In [6]:
housing["yr_renovated"].describe()

count    19957.000000
mean        82.810242
std        398.051608
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max       2015.000000
Name: yr_renovated, dtype: float64

In [ ]:
# Load the holdout dataset
holdout_data = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing_holdout_test_mini.csv', dtype={'id': str})


# seperating id column to match major and minor.
holdout_data['Major'] = holdout_data['id'].str[:-4]
holdout_data['Minor'] = holdout_data['id'].str[-4:].astype(int).astype(str)
# now combining again, this is to get rid of trailing and leading zeros to elimate more duplicates
holdout_data['more'] = holdout_data['Major'].astype(str) + 'm' + holdout_data['Minor'].astype(str)

# now inner joing county data set with training or prediction data set
df2 = pd.read_csv("C://git//ml//housing_project//testcode//merged_data_5.csv", dtype={'more': str})

holdout_data['more'] = holdout_data['more'].astype(str)
df2['more'] = df2['more'].astype(str)

holdout_data = holdout_data.merge(df2, on='more', how='inner')

#print(holdout_data['more'].head(20))
#print(df2['more'].head(20))





# Drop unnecessary columns
X_holdout = holdout_data.drop(["id", "date", "Major_x", "Minor_x", "more", "AcctNbr", "Major_y", "Minor_y", "ZipCode", "SqFtLot"], axis=1)

# Predict using the trained model
y_holdout_pred = model.predict(X_holdout)

# Convert predictions back to original scale
y_holdout_pred_original = np.expm1(y_holdout_pred)

# Create DataFrame with correct format
predictions_df = pd.DataFrame({"price": y_holdout_pred_original})


0       93000m305
1       954160m15
2      733800m150
3       611340m46
4       29131m170
5       728390m36
6     982870m1650
7      734960m230
8      395940m135
9      615020m180
10    788960m2020
11     615290m332
12      387620m70
13     177110m130
14    143270m1230
15    262006m9077
16      205000m20
17    262611m9028
18      886200m75
19      25960m790
Name: more, dtype: object
0     983930m1340
1     983930m1340
2     983930m1330
3     983930m1325
4     983930m1320
5     983930m1315
6     983930m1310
7     983930m1305
8     983930m1300
9     983930m1295
10    983930m1290
11    983930m1285
12    983930m1280
13    983930m1275
14    983930m1270
15    983930m1265
16    983930m1255
17    983930m1250
18    983930m1245
19    983930m1240
Name: more, dtype: object


In [8]:

# Save in the required format
team_name = "team8"  # Replace with your actual team name
filename = f"{team_name}-module3-predictions.csv"
predictions_df.to_csv(filename, index=False)

print(f"Predictions saved as {filename}")

Predictions saved as team8-module3-predictions.csv


In [9]:
from sklearn.metrics import root_mean_squared_error, r2_score
actual_df = pd.read_csv("https://raw.githubusercontent.com/byui-cse/cse450-course/refs/heads/master/data/housing_holdout_test_mini_answers.csv")
rmse = root_mean_squared_error(actual_df, predictions_df)
rmse

84162.24391723484